In [1]:
from functions import *
from variables import *
from datetime import *

###################################################
###           PARAMETRES DES REQUETES          ####
###################################################

# $cCode pour chercher par pays
# $distance pour chercher dans un rayon autour de $city
# $name pour chercher par nom
# $gameId pour filtrer par jeu
# $slug pour filtrer par "slug" (la partie à la fin du lien start.gg)
  # Par exemple dans le lien "https://www.start.gg/tournament/genesis-8/details", le slug est "tournament/genesis-8" ou juste "genesis-8"
# $after et $before pour chercher les évènements à garder avant ou après une certaine date
  # today pour aujourd'hui, get_date(jour,mois,année) pour une autre date
# $page et $perPage pour paginer les évènements

params_get_events={
  "cCode": "FR", 
  "distance": "10km", 
  "city": ville_coord["Montpellier"],
  # "name": "Downtown", 
  "gameId": game_Ids["Super Smash Bros. Ultimate"],
  # "slug": "yggdrasil-1-1",
  # "after": today,
  "after": get_date(1,1,2024),
  "before":  get_date(30,6,2024),
  "before": today,
  "perPage": 500, 
  "page": 1
}

params_standings_seedings={
  # "eventId": 1033454,
  "perPage": 50,
  "page": 1
}


In [2]:

##################################################################
###           RECUPERATION DES EVENEMENTS A ANALYSER          ####
##################################################################
 
singles = get_singles_id(params_get_events, url, headers)
# singles=tournois_sauf_flamingoat
# singles=misstechs


In [3]:
singles

{1165973: 'Smash Downtown #33 - Smash à Lez x Burnout',
 1165971: 'The Roll’Inn#26',
 1165969: "Miss'Tech#60",
 1161681: 'The Roll’Inn#25',
 1131369: 'Yggdrasil #3',
 1157577: 'Smash Downtown #32 - Smash à Lez x Burnout',
 1157571: 'The Roll’Inn#24',
 1157569: "Miss'Tech#59",
 1153371: 'The Roll’Inn#23',
 1153370: "Miss'Tech#58",
 1148869: 'Smash Downtown #31 - Smash à Lez x Burnout',
 1148861: 'The Roll’Inn#22',
 1148859: "Miss'Tech#57",
 1144715: "The Roll'Inn #21",
 1145349: "Miss'Tech#56",
 1140939: 'Smash Downtown #30 - Smash à Lez x Burnout',
 1136292: "Miss'Tech#55",
 1131257: 'Smash Downtown #29 - Smash à Lez x Burnout',
 1131253: "Miss'Tech#54",
 1092534: "The Flamin'Goat #6 - Smash à Lez x MES Esport",
 1126771: "Miss'Tech#53",
 1121814: 'Smash Downtown #28 - Smash à Lez x Burnout',
 1121808: "Miss'Tech#52",
 1117094: "Miss'Tech#51",
 1111736: 'Smash Downtown #27 - Smash à Lez x Burnout',
 1111734: "Miss'Tech#50",
 1107026: "Miss'Tech#49",
 1102045: 'Smash Downtown #26 - Smas

In [4]:
test = {1131369: 'Yggdrasil #3'}

In [28]:
def get_upsets2(events,params,url,headers):
    upsets=[]
    seeding={}
    for event_id in events:
        dict_seeding=get_seeding(event_id, params, url, headers)
        for i in dict_seeding:
            seeding[i[0]]=dict_seeding[i]
        params["eventId"] = str(event_id)

        pages_query = requests.post(url, headers=headers, json={'query': get_sets_pages, 'variables': params}).json()

        pages = pages_query['data']['event']['sets']['pageInfo']['totalPages']

        parametres = {"perPage": 50, "page": pages, "eventId": event_id}

        result = {}
        
        for i in range (1:pages+1):
            print(i)
            parametres = {"perPage": 50, "page": i, "eventId": event_id}
            request = requests.post(url, headers=headers, json={'query': get_sets_nogames, 'variables': parametres}).json()
            result+=request

        for node in request['data']['event']['sets']['nodes']:
            print(node)
            winner_id=node['winnerId']
            for entrant in node['slots']:
                if winner_id!=None:
                    if(entrant['entrant']['id'])==winner_id:
                        winner_name=entrant['entrant']['name']
                    else:
                        loser_name=entrant['entrant']['name']
                else:
                    print("Winner non-renseigné pour le match:", event_id, "set:", node)
            if winner_id!=None:
                if seed_round(seeding[(winner_name)])>seed_round(seeding[(loser_name)]):
                    upsets.append([winner_name,loser_name,spr(seeding[winner_name],seeding[loser_name]),event_id])
    return(upsets)

SyntaxError: invalid syntax (3753985935.py, line 18)

In [27]:
get_upsets2(test,params_standings_seedings,url,headers)

{'perPage': 500, 'page': 1, 'eventId': '1131369'}
{'perPage': 50, 'page': 1, 'eventId': 1131369}
{'winnerId': 16514955, 'slots': [{'entrant': {'id': 16840263, 'name': 'SAN | Draver'}}, {'entrant': {'id': 16514955, 'name': 'Oplon | Mezcaul'}}]}
{'winnerId': 16840263, 'slots': [{'entrant': {'id': 16514955, 'name': 'Oplon | Mezcaul'}}, {'entrant': {'id': 16840263, 'name': 'SAN | Draver'}}]}
{'winnerId': 16940293, 'slots': [{'entrant': {'id': 16700404, 'name': 'Skyflooze'}}, {'entrant': {'id': 16940293, 'name': 'ETY | Irt0zero'}}]}
{'winnerId': 16531477, 'slots': [{'entrant': {'id': 16531477, 'name': 'TLSE | Oxyzz'}}, {'entrant': {'id': 16673646, 'name': 'ECR | LOL TACOS 3'}}]}
{'winnerId': 16840263, 'slots': [{'entrant': {'id': 16840263, 'name': 'SAN | Draver'}}, {'entrant': {'id': 16840596, 'name': 'OX | Naga'}}]}
{'winnerId': 16622812, 'slots': [{'entrant': {'id': 16876440, 'name': 'ETY | SL | KamikazeeSSJ13'}}, {'entrant': {'id': 16622812, 'name': 'SR | LG'}}]}
{'winnerId': 16895652, '

[['SAN | Draver', 'Oplon | Mezcaul', 1, 1131369],
 ['SR | LG', 'ETY | SL | KamikazeeSSJ13', 1, 1131369],
 ['GGO | ECR | DenPuShi', 'RPT | Dasher', 4, 1131369],
 ['ECR | Lorgan', 'VLK | ZeroFK', 2, 1131369],
 ['MYS | ECR | Necross', 'CTS | Goison', 2, 1131369],
 ['BxH | Gohurisson', 'OX | Naga', 1, 1131369],
 ['Her | Victofu', 'POM KSO SSY | TimaChibre', 1, 1131369]]

In [10]:

####################################
###           ANALYSES          ####
####################################

# print_tournaments(singles)
# taille_commu(5,singles,params_standings_seedings,url,headers)
# best_performances(20,singles,params_standings_seedings,url,headers)
# worst_performances(20,singles,params_standings_seedings,url,headers)
# most_regu(10,singles,params_standings_seedings,url,headers)
# least_regu(50,5,singles,params_standings_seedings,url,headers)
# surseed_sousseed(20,5,singles,params_standings_seedings,url,headers)
# print(least_regu(20,10,singles,params_standings_seedings,url,headers))
# top_seed(64,singles,params_standings_seedings,url,headers)
# top_standings(16,singles,params_standings_seedings,url,headers)
# results = get_results(singles,params_standings_seedings,url,headers)
# max_tournois(50,singles,params_standings_seedings,url,headers)
# max_top_x(30,8,singles,params_standings_seedings,url,headers)
# affrontements_freq(50,singles,params_standings_seedings,url,headers)
# bracket_demon(100,5,singles,params_standings_seedings,url,headers)


# count_sets(singles,params_standings_seedings,url,headers)
# count_games(singles,params_standings_seedings,url,headers)
# get_setcount_players('Erakos','Ewanz',singles,params_standings_seedings,url,headers)
# get_setcount_prefix("BaL","NdL", singles, params_standings_seedings, url, headers)
# max_character_usage(500,singles,params_standings_seedings,url,headers)
# min_character_usage(500,singles,params_standings_seedings,url,headers)
# max_character_usage_rate(50,singles,params_standings_seedings,url,headers)
# min_character_usage_rate(50,singles,params_standings_seedings,url,headers)
# upsets=(get_upsets(singles,params_standings_seedings,url,headers))
# for i in upsets:
#     for j in range(len(i)):
#         print(i[j])
# count_upsets(singles,params_standings_seedings,url,headers, False)
# biggest_upsets(50,test,params_standings_seedings,url,headers)
# max_upsets_realises(20,singles,params_standings_seedings,url,headers)
# max_upsets_subis(20,singles,params_standings_seedings,url,headers)
# max_upsets_subis_par_defaite(20,singles,params_standings_seedings,url,headers)
# min_upsets_subis(500,singles,params_standings_seedings,url,headers)

# placements=get_player_placement("El Tépe",singles,params_standings_seedings,url,headers)
# n=0
# for event in placements:
#   if placements[event] == 9:
#     n+=1
# print("El Tépe a fini 9ème",n,"fois en", len(placements),"participations (",(n/len(placements))*100,"%)")

1) Dominique Tonrétro: 1.1666666666666667
2) Archi Bald: 0.8333333333333334
3) Natsu34: 0.8
4) Bizaute: 0.7586206896551724
5) Magicana: 0.75
6) MatrX: 0.6666666666666666
7) Zone-Onze: 0.6363636363636364
8) Osca: 0.625
9) Skyflooze: 0.6
10) ThomasSwitch: 0.6
11) Dr.Tea: 0.6
12) Poade: 0.48
13) Tyzoute: 0.4782608695652174
14) Maulisu: 0.4666666666666667
15) Nuamus: 0.45454545454545453
16) GreGrenier: 0.4444444444444444
17) Zelume: 0.4222222222222222
18) Poutre: 0.4090909090909091
19) Lenyous: 0.4
20) KSO: 0.4

1) Juartz: -1.8333333333333333
2) Yas As1: -1.75
3) Flynn: -1.393939393939394
4) Totoledino: -1.2
5) Goison: -1.0857142857142856
6) Elkiah: -1.0
7) Orik: -0.875
8) UnCalin: -0.6666666666666666
9) inori: -0.625
10) Erakos: -0.5416666666666666
11) Mr.Grizzly: -0.5
12) Limul: -0.5
13) El Tépe: -0.4117647058823529
14) helryzers: -0.4
15) OSLOW: -0.4
16) Kajou: -0.391304347826087
17) Virgo: -0.3333333333333333
18) Trozer176: -0.3333333333333333
19) satho: -0.2857142857142857
20) Reveяse

In [ ]:
params_standings_seedings={
  "perPage": 100,
  "page": 1
}

In [ ]:
get_sets="""
query get_sets($eventId: ID!, $page: Int!, $perPage: Int!) {
  event(id: $eventId) {
    id
    name
    sets(
      page: $page
      perPage: $perPage
      sortType: STANDARD
    ) {
      nodes {
        winnerId
        slots {
          entrant {
            id
            name
          }
        }
        games{
            winnerId
        }
      }
    }
  }
}
"""

In [ ]:
def get_upsets(events,params,url,headers):
    upsets=[]
    seeding={}
    for event_id in events:
        dict_seeding=get_seeding(event_id, params, url, headers)
        for i in dict_seeding:
            seeding[i[0]]=dict_seeding[i]
        params["eventId"] = str(event_id)
        response = requests.post(url, headers=headers, json={'query': get_sets_nogames, 'variables': params})
        request = response.json()
        print(request)
        for node in request['data']['event']['sets']['nodes']:
            print(node)
            winner_id=node['winnerId']
            for entrant in node['slots']:
                if winner_id!=None:
                    if(entrant['entrant']['id'])==winner_id:
                        winner_name=entrant['entrant']['name']
                    else:
                        loser_name=entrant['entrant']['name']
                else:
                    print("Winner non-renseigné pour le match:", event_id, "set:", node)
            if winner_id!=None:
                if seed_round(seeding[(winner_name)])>seed_round(seeding[(loser_name)]):
                    upsets.append([winner_name,loser_name,spr(seeding[winner_name],seeding[loser_name]),event_id])
    return(upsets)

In [ ]:
get_upsets(test, params_standings_seedings, url, headers)

{'data': {'event': {'id': 1131369, 'name': 'Yggdrasil #3', 'sets': {'nodes': [{'winnerId': 16514955, 'slots': [{'entrant': {'id': 16840263, 'name': 'SAN | Draver'}}, {'entrant': {'id': 16514955, 'name': 'Oplon | Mezcaul'}}]}, {'winnerId': 16840263, 'slots': [{'entrant': {'id': 16514955, 'name': 'Oplon | Mezcaul'}}, {'entrant': {'id': 16840263, 'name': 'SAN | Draver'}}]}, {'winnerId': 16940293, 'slots': [{'entrant': {'id': 16700404, 'name': 'Skyflooze'}}, {'entrant': {'id': 16940293, 'name': 'ETY | Irt0zero'}}]}, {'winnerId': 16531477, 'slots': [{'entrant': {'id': 16531477, 'name': 'TLSE | Oxyzz'}}, {'entrant': {'id': 16673646, 'name': 'ECR | LOL TACOS 3'}}]}, {'winnerId': 16840263, 'slots': [{'entrant': {'id': 16840263, 'name': 'SAN | Draver'}}, {'entrant': {'id': 16840596, 'name': 'OX | Naga'}}]}, {'winnerId': 16622812, 'slots': [{'entrant': {'id': 16876440, 'name': 'ETY | SL | KamikazeeSSJ13'}}, {'entrant': {'id': 16622812, 'name': 'SR | LG'}}]}, {'winnerId': 16895652, 'slots': [{'en

[['SAN | Draver', 'Oplon | Mezcaul', 1, 1131369],
 ['SR | LG', 'ETY | SL | KamikazeeSSJ13', 1, 1131369],
 ['GGO | ECR | DenPuShi', 'RPT | Dasher', 4, 1131369],
 ['ECR | Lorgan', 'VLK | ZeroFK', 2, 1131369],
 ['MYS | ECR | Necross', 'CTS | Goison', 2, 1131369],
 ['BxH | Gohurisson', 'OX | Naga', 1, 1131369],
 ['Her | Victofu', 'POM KSO SSY | TimaChibre', 1, 1131369],
 ['SAN | Draver', 'Oplon | Mezcaul', 1, 1131369],
 ['Skyflooze', 'IZI | Eko', 10, 1131369],
 ['CTS | Mephy', 'Original', 1, 1131369],
 ['ORBR | | Froa & Nana', 'NMS | Saka', 2, 1131369],
 ['ETY ECR | | Pacha', 'TR NMS | Zenatti', 7, 1131369],
 ['NdL | Threllox', 'CRC | José le plombier', 2, 1131369],
 ['BaL | Enzo', 'CTS | Goison', 5, 1131369],
 ['Tyzoute', 'Mr.Grizzly', 1, 1131369],
 ['FF | Kyuu', 'SR | Speedroide', 2, 1131369],
 ['AFT | TLS | Maskime', 'LT | Plouf', 1, 1131369],
 ['AMU | RED', 'TR NMS | Zenatti', 9, 1131369]]